In [1]:
from scipy.special import softmax
from dotenv import load_dotenv
import json
import os

load_dotenv()

True

In [2]:
prompt = ''''You should answer the following question with Yes or No only.

question: Is this an analysis that covers the most relevant topics of the document? 

### analysis: 
Petrobras Looks to Increase Production Through Tenders Under Production Sharing Regime   
Petrobras announced its interest in exercising its right of first refusal for the Jaspe block in the Campos Basin, a move aligned with the company's strategy of focusing on profitable assets and replenishing oil and gas reserves. This decision comes under the Production Sharing Regime, which aims to increase investment and boost production in the Brazilian oil and gas sector. 
While this move by Petrobras aligns with their stated goals and strategies, it's imperative to closely monitor how this affects the company's performance and returns. The success of the Jaspe block will determine the feasibility of future investments under the Production Sharing Regime. 

### document:
Petrobras expresses interest in area under the Production Sharing regime
Rio de Janeiro, January 24, 2024 - Petróleo Brasileiro S.A. - Petrobras informs that it expressed today to the National Energy Policy Council (CNPE) its interest in the right of first refusal in a block to be tendered in the Permanent Offer System, under the Production Sharing Regime, under the terms of Law 12,351/2010 and Federal Decree 9,041/2017.
Petrobras approved the expression of interest in the right of first refusal in the Jaspe block, located in the Campos Basin, considering the parameters disclosed in CNPE Resolution No. 11, of December 20, 2023, published on December 27, 2023.
pression of interest is in line with the E&P strategy set out in SP 24-28+, focusing on profitable assets and replenishing oil and gas reserves.
Material facts on the subject will be disclosed to the market in due course.The ex
'''

In [3]:
an = '''Petrobras Looks to Increase Production Through Tenders Under Production Sharing Regime   
Petrobras announced its interest in exercising its right of first refusal for the Jaspe block in the Campos Basin, a move aligned with the company's strategy of focusing on profitable assets and replenishing oil and gas reserves. This decision comes under the Production Sharing Regime, which aims to increase investment and boost production in the Brazilian oil and gas sector. 
While this move by Petrobras aligns with their stated goals and strategies, it's imperative to closely monitor how this affects the company's performance and returns. The success of the Jaspe block will determine the feasibility of future investments under the Production Sharing Regime. '''

In [7]:
dc = '''Petrobras expresses interest in area under the Production Sharing regime
Rio de Janeiro, January 24, 2024 - Petróleo Brasileiro S.A. - Petrobras informs that it expressed today to the National Energy Policy Council (CNPE) its interest in the right of first refusal in a block to be tendered in the Permanent Offer System, under the Production Sharing Regime, under the terms of Law 12,351/2010 and Federal Decree 9,041/2017.
Petrobras approved the expression of interest in the right of first refusal in the Jaspe block, located in the Campos Basin, considering the parameters disclosed in CNPE Resolution No. 11, of December 20, 2023, published on December 27, 2023.
pression of interest is in line with the E&P strategy set out in SP 24-28+, focusing on profitable assets and replenishing oil and gas reserves.
Material facts on the subject will be disclosed to the market in due course.The ex'''

In [3]:
import openai
from openai import OpenAI
client = OpenAI()
openai.key = os.environ['OPENAI_API_KEY']

completion = client.chat.completions.create(
  model="gpt-4-turbo-preview",
  messages=[
    {"role": "user", "content": f'{prompt}'}
  ],
  logprobs=True,
  top_logprobs=5,
  temperature=0,
  max_tokens=1
)

print(completion.choices[0].message)
print(completion.choices[0].logprobs)

ChatCompletionMessage(content='Yes', role='assistant', function_call=None, tool_calls=None)
ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.008350736, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.008350736), TopLogprob(token='No', bytes=[78, 111], logprob=-4.789601), TopLogprob(token=' Yes', bytes=[32, 89, 101, 115], logprob=-15.57085), TopLogprob(token='YES', bytes=[89, 69, 83], logprob=-18.367725), TopLogprob(token='yes', bytes=[121, 101, 115], logprob=-18.398975)])])


In [4]:
completion.choices[0].message

ChatCompletionMessage(content='Yes', role='assistant', function_call=None, tool_calls=None)

In [14]:
completion.choices[0].logprobs.content[0].logprob

-0.008350736

In [15]:
probs = completion.choices[0].logprobs.content[0].top_logprobs

In [22]:
def save_json(name, tokens, path=""):
    data = {
        "src": name,
        "tokens": tokens
    }
    
    filename = f"`{path}{name}.json"
    
    # Salvando o dicionário como um arquivo JSON
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

In [23]:
vetor = []
for prob in probs:
    aux = [prob.token, prob.logprob]
    vetor.append(aux)

save_json("teste", vetor)

In [30]:
completion.choices[0].logprobs

ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.008350736, top_logprobs=[TopLogprob(token='Yes', bytes=[89, 101, 115], logprob=-0.008350736), TopLogprob(token='No', bytes=[78, 111], logprob=-4.789601), TopLogprob(token=' Yes', bytes=[32, 89, 101, 115], logprob=-15.57085), TopLogprob(token='YES', bytes=[89, 69, 83], logprob=-18.367725), TopLogprob(token='yes', bytes=[121, 101, 115], logprob=-18.398975)])])

In [4]:
PROMPTS = {
    "relevance": "You should answer the following question with Yes or No only.\nIs this an analysis that covers the most relevant topics of the document?\n\n###analysis:\n{analysis}\n\n###document:\n{document}"
}

In [5]:
def replace_variables_in_prompt(key, **variables):

    if key in PROMPTS:
        prompt_template = PROMPTS[key]
        return prompt_template.format(**variables)
    else:
        return "Chave de prompt não encontrada."

In [8]:
a = replace_variables_in_prompt("relevance", analysis=an, document=dc)

print(a)

You should answer the following question with Yes or No only.
Is this an analysis that covers the most relevant topics of the document?

###analysis:
Petrobras Looks to Increase Production Through Tenders Under Production Sharing Regime   
Petrobras announced its interest in exercising its right of first refusal for the Jaspe block in the Campos Basin, a move aligned with the company's strategy of focusing on profitable assets and replenishing oil and gas reserves. This decision comes under the Production Sharing Regime, which aims to increase investment and boost production in the Brazilian oil and gas sector. 
While this move by Petrobras aligns with their stated goals and strategies, it's imperative to closely monitor how this affects the company's performance and returns. The success of the Jaspe block will determine the feasibility of future investments under the Production Sharing Regime. 

###document:
Petrobras expresses interest in area under the Production Sharing regime
Rio 

In [25]:
PROMPTS['relevance']

'You should answer the following question with Yes or No only.\nIs this an analysis that covers the most relevant topics of the document?\n\n###analysis:{analysis}\n\n###document:{document}'

In [10]:
a = softmax([-0.007846756, -4.851597])

In [12]:
a[1]

0.00781588689711749

In [20]:
prompt2 = '''You will be given one analysis written based on a document.

Your task is to rate the analysis on one metric.

Please make sure you read and understand these instructions carefully. Please keep this document open while reviewing, and refer to it as needed.


Evaluation Criteria:

Fluency (1-3): the quality of the analysis in terms of grammar, spelling, punctuation, word choice, and sentence structure.

- 1: Poor. The analysis has many errors that make it hard to understand or sound unnatural.
- 2: Fair. The analysis has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
- 3: Good. The analysis has few or no errors and is easy to read and follow.


Example:

Analysis:
Petrobras Looks to Increase Production Through Tenders Under Production Sharing Regime   
Petrobras announced its interest in exercising its right of first refusal for the Jaspe block in the Campos Basin, a move aligned with the company's strategy of focusing on profitable assets and replenishing oil and gas reserves. This decision comes under the Production Sharing Regime, which aims to increase investment and boost production in the Brazilian oil and gas sector. 
While this move by Petrobras aligns with their stated goals and strategies, it's imperative to closely monitor how this affects the company's performance and returns. The success of the Jaspe block will determine the feasibility of future investments under the Production Sharing Regime. 


Evaluation Form (scores ONLY):

- Fluency (1-3):

'''

In [21]:
completion = client.chat.completions.create(
  model="gpt-4-turbo-preview",
  messages=[
    {"role": "user", "content": f'{prompt2}'}
  ],
  logprobs=True,
  top_logprobs=5,
  temperature=0,
  max_tokens=1
)

print(completion.choices[0].message)
print(completion.choices[0].logprobs)

ChatCompletionMessage(content='3', role='assistant', function_call=None, tool_calls=None)
ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='3', bytes=[51], logprob=-7.89631e-07, top_logprobs=[TopLogprob(token='3', bytes=[51], logprob=-7.89631e-07), TopLogprob(token='Fl', bytes=[70, 108], logprob=-14.625001), TopLogprob(token='Evaluation', bytes=[69, 118, 97, 108, 117, 97, 116, 105, 111, 110], logprob=-15.437501), TopLogprob(token='2', bytes=[50], logprob=-17.109375), TopLogprob(token='Score', bytes=[83, 99, 111, 114, 101], logprob=-17.78125)])])
